In [64]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import tabula

## Importing Data

### Bus Ridership and Stop Usage Data

sources:
- https://data.wprdc.org/dataset/prt-transit-stop-usage
- https://data.wprdc.org/dataset/prt-monthly-average-ridership-by-route

In [67]:
ridership_df = pd.read_csv("Data/avg_ridership.csv")
ridership_df.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4


In [68]:
stopuse_df = pd.read_csv("Data/wprdc_stop_data.csv")
stopuse_df.head()

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,datekey,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,202001,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00


### Weather Data

In [70]:
## Monthly average temperatures in Pittsburgh, PA
temp_url = "https://www.weather.gov/media/pbz/records/histemp.pdf"
temp_df = tabula.read_pdf(temp_url, pages = 'all')[2]

In [71]:
cols = ['Year', 'January', 'February', 'March', 
        'April', 'May', 'June', 'July', 'August', 
        'September', 'October', 'November', 'December', 'Annual']
temp_df.columns = cols

In [72]:
temps_2024 = [2024, 33.0, 39.5, 46.6, 56.9, 66.5, 72.8, 76.5, 73.8, 69.4, 57.0, 47.6, 36.6, 56.4]
temps_2025 = [2025, 24.1, 31.7, 47.4] + [np.nan for i in range(len(cols)-3)]

temps_2024_df = dict(zip(cols, temps_2024))
temps_2024_df = pd.DataFrame(temps_2024_df, index = [1])

temps_2025_df = dict(zip(cols, temps_2025))
temps_2025_df = pd.DataFrame(temps_2025_df, index = [1])

In [73]:
temp_df = pd.concat([temp_df, temps_2024_df, temps_2025_df])
temp_df.reset_index(drop = True, inplace = True)
temp_df.head()

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual
0,1975,32.6,32.1,36.3,44.3,63.0,57.8,72.8,73.0,58.8,53.3,46.3,32.9,51.1
1,1976,23.5,37.2,45.2,50.6,55.6,68.4,67.4,65.3,59.9,45.9,33.1,23.9,48.0
2,1977,11.4,26.9,43.7,50.8,63.0,63.8,71.8,68.1,64.7,50.5,45.6,31.1,49.3
3,1978,22.6,20.9,36.9,51.0,60.2,69.4,73.0,71.4,66.2,49.1,43.0,32.7,49.7
4,1979,21.4,18.0,43.1,49.7,59.1,67.7,70.3,69.6,63.4,50.9,44.7,34.6,49.4


In [74]:
temp_df = temp_df.melt(id_vars = ['Year'], 
                       var_name = 'Month', 
                       value_name = 'avg_temp')
temp_df.head()

,Year,Month,avg_temp
0,1975,January,32.6
1,1976,January,23.5
2,1977,January,11.4
3,1978,January,22.6
4,1979,January,21.4


In [75]:
temp_df = temp_df[temp_df['Month'] != 'Annual'].copy()
temp_df = temp_df[temp_df['Year'] != '30 yr'].copy()

d = dict((v,k) for k,v in zip(range(1, 13), temp_df.Month.unique()))
temp_df['month_index'] = temp_df['Month'].map(d)
temp_df['year_month'] = temp_df['Year'].astype(str) + temp_df['month_index'].astype(str).str.zfill(2)

temp_df

,Year,Month,avg_temp,month_index,year_month
0,1975,January,32.6,1,197501
1,1976,January,23.5,1,197601
2,1977,January,11.4,1,197701
3,1978,January,22.6,1,197801
4,1979,January,21.4,1,197901
...,...,...,...,...,...
607,2021,December,40.9,12,202112
608,2022,December,33.0,12,202212
609,2023,December,42.7,12,202312
610,2024,December,36.6,12,202412


In [76]:
## Monthly total precipitation (inches) in Pittsburgh, PA
precip_url = "https://www.weather.gov/media/pbz/records/hisprec.pdf"
precip_df = tabula.read_pdf(precip_url, pages = 'all')[2]

In [77]:
precip_df.columns = ['Year', 'January', 'February', 'March', 
                     'April', 'May', 'June', 'July', 'August', 
                     'September', 'October', 'November', 'December', 'Annual']
precip_df.head()

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual
0,1964,2.55,1.73,4.96,7.61,1.77,3.84,4.48,1.79,0.74,1.42,2.74,4.26,37.89
1,1965,3.84,2.98,3.16,1.79,1.21,2.31,1.82,3.26,4.07,2.82,2.35,0.63,30.24
2,1966,4.52,3.23,1.88,3.73,2.76,1.72,2.70,5.13,1.92,1.38,3.39,1.70,34.06
3,1967,1.06,2.54,6.10,4.41,5.21,0.90,4.54,2.67,1.61,2.05,3.07,2.22,36.38
4,1968,2.83,0.79,4.53,2.33,6.36,2.38,2.36,3.97,3.08,2.13,2.07,3.24,36.07


In [78]:
precip_df = precip_df.melt(id_vars = ['Year'], 
                           var_name = 'Month', 
                           value_name = 'total_precip')
precip_df.head()

,Year,Month,total_precip
0,1964,January,2.55
1,1965,January,3.84
2,1966,January,4.52
3,1967,January,1.06
4,1968,January,2.83


In [79]:
precip_df = precip_df[precip_df['Month'] != 'Annual'].copy()
precip_df = precip_df[precip_df['Year'] != '30 yr'].copy()

d = dict((v,k) for k,v in zip(range(1, 13), precip_df.Month.unique()))
precip_df['month_index'] = precip_df['Month'].map(d)
precip_df['year_month'] = precip_df['Year'].astype(str) + precip_df['month_index'].astype(str).str.zfill(2)

precip_df

,Year,Month,total_precip,month_index,year_month
0,1964,January,2.55,1,196401
1,1965,January,3.84,1,196501
2,1966,January,4.52,1,196601
3,1967,January,1.06,1,196701
4,1968,January,2.83,1,196801
...,...,...,...,...,...
750,2021,December,3.45,12,202112
751,2022,December,1.76,12,202212
752,2023,December,2.80,12,202312
753,2024,December,2.71,12,202412


### ACS Population Data

In [81]:
## TODO

## Merge Data

### Add weather data to bus data

In [84]:
stopuse_df['datekey'] = stopuse_df['datekey'].astype(str)
stopuse_df = stopuse_df.merge(precip_df[['year_month', 'total_precip']], how = 'left', left_on = 'datekey', right_on = 'year_month')
stopuse_df = stopuse_df.merge(temp_df[['year_month', 'avg_temp']], how = 'left', left_on = 'datekey', right_on = 'year_month')
stopuse_df.drop(columns = ['year_month_x', 'year_month_y'], inplace = True)
stopuse_df.head()

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,...,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs,total_precip,avg_temp
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00,5.34,69.0
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00,5.34,69.0
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05,5.34,69.0
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00,5.34,69.0
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00,3.69,35.2


In [85]:
ridership_df['year_month'] = ridership_df['year_month'].astype(str)
ridership_df = ridership_df.merge(precip_df[['year_month', 'total_precip']], how = 'left', on = 'year_month')
ridership_df = ridership_df.merge(temp_df[['year_month', 'avg_temp']], how = 'left', on = 'year_month')
ridership_df.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count,total_precip,avg_temp
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4,3.54,34.6
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4,3.54,34.6
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4,3.54,34.6
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4,3.54,34.6
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4,3.54,34.6


### Add ZIP code to bus data

In [87]:
## TODO

## Output Data

In [89]:
stopuse_df.to_csv("Data/stop_usage.csv", index = False)
ridership_df.to_csv("Data/ridership.csv", index = False)

In [90]:
#demo_df.to_csv("Data/zip_demo.csv", index = False)
#commute_df.to_csv("Data/zip_commute.csv", index = False)